In [5]:
import pandas as pd
import RIfunctions as ri

In [6]:
application_df = pd.read_csv('input_file_1.csv', sep = ',', index_col=0)
accept_df = pd.read_csv('input_file_2.csv', sep = ',', index_col=0)

In [8]:
application_df['rejected'].value_counts()

1    14739886
0      814643
Name: rejected, dtype: int64

In [9]:
rejects = application_df[application_df['rejected'] == 1]

In [10]:
accept_df.shape

(814643, 21)

In [11]:
rejects

,dti,emp_length,issue_d,loan_amnt,purpose,rejected
0,4.21,5.0,2016-07-01,4000.0,major_purchase,1
1,0.39,0.0,2016-07-01,20000.0,debt_consolidation,1
2,42.38,5.0,2016-07-01,1000.0,renewable_energy,1
3,5.98,5.0,2016-07-01,15000.0,home_improvement,1
4,29.14,0.0,2016-07-01,8500.0,debt_consolidation,1
...,...,...,...,...,...,...
16131467,16.08,0.0,2015-12-31,7500.0,major_purchase,1
16131468,2.34,5.0,2015-12-31,12000.0,major_purchase,1
16131469,17.00,0.0,2015-12-31,30000.0,major_purchase,1
16131470,0.60,0.0,2015-12-31,2000.0,debt_consolidation,1


In [12]:
ri_df = pd.concat([accept_df, rejects], axis=0, ignore_index=True)

In [13]:
ri_df['charged_off'].value_counts()

0.0    646618
1.0    168025
Name: charged_off, dtype: int64

In [14]:
ri_df.loc[ri_df['rejected'] == 1, 'charged_off'] = 1

In [15]:
ri_df['charged_off'].value_counts()

1.0    14907911
0.0      646618
Name: charged_off, dtype: int64

In [16]:
ri_df.drop('rejected', axis=1, inplace=True)

In [17]:
nan_cols = [i for i in ri_df.columns if ri_df[i].isna().any()]
nan_cols = [e for e in nan_cols if e not in ('emp_length', 'purpose')]

In [18]:
ri_df.drop(nan_cols, axis=1, inplace=True)

In [19]:
ri_df = ri.purposeCleaning(ri_df)

c:\Users\joshy\OneDrive - Heriot-Watt University\Documents\Heriot Watt\Year 2\MSc Project\Reject-Inference\RIfunctions.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['purpose'] = df['purpose'].replace(to_replace)


In [23]:
ri_df.to_csv('RI_Accept.csv')